In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from pathlib import Path
import requests

%matplotlib inline

In [2]:
!pwd

/Users/josephbarry/Desktop/FinTech_Class/3rd_Project/FinTech_Portfolio_Dash/JGB_Notebook


In [3]:
csvpath = Path("AMD.csv")

df = pd.read_csv(csvpath, index_col='Date', infer_datetime_format=True, parse_dates=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-08-16,6.36,6.43,6.18,6.34,6.34,16355700
2011-08-17,6.41,6.47,6.08,6.14,6.14,15248500
2011-08-18,5.90,5.96,5.64,5.79,5.79,27200300
2011-08-19,5.72,6.09,5.71,5.98,5.98,37995200
2011-08-22,6.13,6.18,5.95,6.05,6.05,18433500


In [4]:
df = df[['Close']]
df

,Close
Date,
2011-08-16,6.340000
2011-08-17,6.140000
2011-08-18,5.790000
2011-08-19,5.980000
2011-08-22,6.050000
...,...
2021-08-09,107.580002
2021-08-10,106.480003
2021-08-11,107.680000


In [5]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window- 1):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [7]:
window_size = 100
feature_column = 0 # why 0?
target_column = 0 # Why 0?

X, y = window_data(df, window_size, feature_column, target_column)

print (f"X sample values:\n{X[:3]} \n")
print (f"y sample values:\n{y[:3]}")

X sample values:
[[6.34 6.14 5.79 5.98 6.05 6.5  6.34 6.37 6.49 6.83 6.84 6.83 6.68 6.32
  6.27 6.76 6.81 6.52 6.71 6.96 7.21 7.34 7.2  6.92 6.74 6.48 6.12 6.17
  6.4  6.49 6.15 5.31 5.08 4.53 4.72 4.83 4.85 4.73 4.84 4.86 4.93 4.97
  4.92 4.67 4.83 4.73 4.54 4.72 5.06 4.91 5.1  5.54 5.94 5.83 5.3  5.46
  5.73 5.67 5.68 5.75 5.38 5.66 5.95 5.79 5.76 5.68 5.44 5.47 5.42 5.3
  5.05 4.99 5.3  5.32 5.69 5.7  5.65 5.76 5.66 5.72 5.47 5.54 5.3  5.18
  5.07 5.04 5.13 4.86 5.19 5.08 5.45 5.41 5.38 5.28 5.34 5.4  5.48 5.46
  5.46 5.43]
 [6.14 5.79 5.98 6.05 6.5  6.34 6.37 6.49 6.83 6.84 6.83 6.68 6.32 6.27
  6.76 6.81 6.52 6.71 6.96 7.21 7.34 7.2  6.92 6.74 6.48 6.12 6.17 6.4
  6.49 6.15 5.31 5.08 4.53 4.72 4.83 4.85 4.73 4.84 4.86 4.93 4.97 4.92
  4.67 4.83 4.73 4.54 4.72 5.06 4.91 5.1  5.54 5.94 5.83 5.3  5.46 5.73
  5.67 5.68 5.75 5.38 5.66 5.95 5.79 5.76 5.68 5.44 5.47 5.42 5.3  5.05
  4.99 5.3  5.32 5.69 5.7  5.65 5.76 5.66 5.72 5.47 5.54 5.3  5.18 5.07
  5.04 5.13 4.86 5.19 5.08 5.45 5.41

In [8]:
split = int(0.6 * len(X))

X_train = X[: split]
X_test = X[split:]

y_train = y[: split]
y_test = y[split:]

In [9]:
# Importing the MinMaxScaler from sklearn
from sklearn.preprocessing import MinMaxScaler

x_train_scaler = MinMaxScaler()
#x_test_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
#y_test_scaler = MinMaxScaler()

# Fit the scaler for the training Data
x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

# Scale the training data
X_train = x_train_scaler.transform(X_train)
y_train = y_train_scaler.transform(y_train)

# Fit the scaler for the testing Data
#x_test_scaler.fit(X_test)
#y_test_scaler.fit(y_test)

# Scale the y_test data
X_test = x_train_scaler.transform(X_test)
y_test = y_train_scaler.transform(y_test)

In [10]:
# Reshape the features data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [11]:
# Importing required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [12]:
# Define the LSTM RNN model.
model = Sequential()

# Initial model setup
number_units = window_size
# 20% of neurons to be dropped out
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

In [13]:
model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 100)          40800     
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1

In [ ]:
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=100, verbose=1)

Epoch 1/20
15/15 [==============================] - 1s 67ms/step - loss: 0.0455
Epoch 2/20
15/15 [==============================] - 1s 67ms/step - loss: 0.0253
Epoch 3/20
15/15 [==============================] - 1s 68ms/step - loss: 0.0075
Epoch 4/20
15/15 [==============================] - 1s 66ms/step - loss: 0.0254
Epoch 5/20
15/15 [==============================] - 1s 67ms/step - loss: 0.0220
Epoch 6/20
15/15 [==============================] - 1s 72ms/step - loss: 0.0222
Epoch 7/20
15/15 [==============================] - 1s 70ms/step - loss: 0.0055
Epoch 8/20
15/15 [==============================] - 1s 68ms/step - loss: 0.0083
Epoch 9/20
15/15 [==============================] - 1s 68ms/step - loss: 0.0095
Epoch 10/20
15/15 [==============================] - 1s 67ms/step - loss: 0.0024
Epoch 11/20
15/15 [==============================] - 1s 69ms/step - loss: 0.0031
Epoch 12/20
15/15 [==============================] - 1s 68ms/step - loss: 0.0074
Epoch 13/20
 7/15 [=============>....

In [ ]:
model.evaluate(X_test, y_test, verbose=0)

In [ ]:
predicted = model.predict(X_test)